In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "autoencoders"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
np.random.seed(42)

def generate_3d_data(m, w1=0.1, w2=0.3, noise=0.1):
    angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
    data = np.empty((m, 3))
    data[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
    data[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
    data[:, 2] = data[:, 0] * w1 + data[:, 1] * w2 + noise * np.random.randn(m)
    return data

X_train = generate_3d_data(60)
X_train = X_train - X_train.mean(axis=0, keepdims=0)

17.2 과소완전 선형  오토인코더로 PCA 수행하기

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)


encoder = keras.models.Sequential([keras.layers.Dense(2, input_shape=[3])])
decoder = keras.models.Sequential([keras.layers.Dense(3,input_shape=[2])])

autoencoder = keras.models.Sequential([encoder,decoder])

#pca 는 활성화함수 없이 선형임
autoencoder.compile(loss="mse", optimizer = keras.optimizers.SGD(lr=1.5))

In [ ]:

history = autoencoder.fit(X_train,X_train,epochs=20)

#은닉층 출력 (앞부분 이용)
codings = encoder.predict(X_train)

In [ ]:
fig  = plt.figure(figsize=(4,3))
plt.plot(codings[:,0], codings[:,1], "b.")
plt.xlabel("$z_1$", fontsize=18)
plt.ylabel("$z_2$",fontsize=18,rotation=0)
plt.grid(True)
plt.show()

# 17.3 적층 오토인코더

Fashion MNIST

In [ ]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()

X_train_full = X_train_full.astype(np.float32)/255
X_test  =X_test.astype(np.float32)/255

X_train,X_valid = X_train_full[5000:],X_train_full[:5000]
y_train,y_valid = y_train_full[5000:],y_train_full[:5000]


이진분류로 학습 가능.

회귀보다 빠름.

픽셀이 흑백인가 아닌가를 맞추는 문제로 이진분류 학습

In [ ]:
stacked_encoder = keras.models.Sequential([
                                           keras.layers.Flatten(input_shape=[28,28]),
                                           keras.layers.Dense(100,activation="selu"),
                                           keras.layers.Dense(30,activation="selu")
])
stacked_decoder = keras.models.Sequential([
                                           keras.layers.Dense(100,activation="selu", input_shape=[30]),
                                           keras.layers.Dense(28*28,activation="sigmoid"),
                                           keras.layers.Reshape([28,28])
])

stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])
stacked_ae.compile(loss= keras.losses.binary_crossentropy,
                   optimizer=keras.optimizers.SGD(lr=1.5))

history = stacked_ae.fit(X_train,X_train, epochs=10,
                         validation_data = (X_valid,X_valid))

### 17.3.2 재구성 시각화

오토인코더의 성능확인을 위해 입력과 출력을 비교

In [ ]:
def plot_image(image):
  plt.imshow(image,cmap="binary")
  plt.axis("off")

def show_reconstructions(model, n_images=5):
  #코딩(중간 특성맵)을 보는것이 아니라 재구성을 위해 모델수준에서 예 
  reconstructions = model.predict(X_valid[:n_images])
  fig = plt.figure(figsize=(n_images*1.5,3))
  for image_index in range(n_images):
    plt.subplot(2,n_images,1+image_index)
    plot_image(X_valid[image_index])
    plt.subplot(2,n_images,1+n_images+image_index)
    plot_image(reconstructions[image_index])


show_reconstructions(stacked_ae)

### 17.3.3 패션 MNIST 데이터셋 시각화

훈련한 모델을 바탕으로 Fashion mnist 데이터셋 차원축소

In [ ]:
from sklearn.manifold import TSNE

X_valid_compressed = stacked_encoder.predict(X_valid)
tsne  = TSNE()
X_valid_2d = tsne.fit_transform(X_valid_compressed)

시각화

In [ ]:
plt.scatter(X_valid_2d[:,0],X_valid_2d[:,1], c=y_valid, s=10 , cmap="tab10")

### 17.3.5 가중치 묶기

인코더와 디코더가 대칭일때 마주보는 가중치를 묶어서 가중치를 반으로 줄임.

층을 입력으로 받아 전치된 가중치를 사용하는 사용자정의층

In [ ]:
class DenseTranspose(keras.layers.Layer):
  def __init__(self,dense, activation=None, **kwargs):
    super().__init__(**kwargs)
    self.dense = dense
    self.activation=keras.activations.get(activation)

  def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=[self.dense.input_shape[-1]],
                                      initializer="zeros")
        super().build(batch_input_shape)


  def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(z + self.biases)

In [ ]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

dense_1 = keras.layers.Dense(100,activation="selu")
dense_2 = keras.layers.Dense(30,activation="selu")

tied_encoder =keras.models.Sequential([
                                       keras.layers.Flatten(input_shape=[28,28]),
                                       dense_1,
                                       dense_2
])

tied_decoder = keras.models.Sequential([
                                       DenseTranspose(dense_2,activation="selu"),
                                       DenseTranspose(dense_1,activation="sigmoid"),
                                       keras.layers.Reshape([28,28])
])

tied_ae = keras.models.Sequential([tied_encoder, tied_decoder])

tied_ae.compile(loss=keras.losses.binary_crossentropy,
                optimizer= keras.optimizers.SGD(lr=1.5),
                metrics =[rounded_accuracy])

In [ ]:
history = tied_ae.fit(X_train, X_train, epochs=10,
                      validation_data=(X_valid, X_valid))

In [ ]:
show_reconstructions(tied_ae)

# 17.4 합성곱 오토인코더

이미지를 위한 오토인코더.

디코더가 거꾸로(이미지가 커지게) 동작해야하므로 , 전치합성곱(conv2dtranspose) 혹은 업샘플링 사용

전치합성곱: 0을 추가해 스트라이드 만큼 늘림(반대)

In [ ]:
conv_encoder = keras.models.Sequential([
                                        keras.layers.Reshape([28,28,1],input_shape=[28,28]),
                                        keras.layers.Conv2D(16,kernel_size=3,padding="same",activation="relu"),
                                        keras.layers.MaxPool2D(pool_size=2),
                                        keras.layers.Conv2D(32,kernel_size=3,padding="same",activation="relu"),
                                        keras.layers.MaxPool2D(pool_size=2),
                                        keras.layers.Conv2D(64,kernel_size=3,padding="same",activation="relu"),
                                        keras.layers.MaxPool2D(pool_size=2),
])


conv_decoder = keras.models.Sequential([
                                        #전치 합성곱층을 사용해 특성맵 크기를 다시 늘림.
                                        keras.layers.Conv2DTranspose(32,kernel_size=3,strides=2,padding="valid",
                                                                     activation="relu"),
                                        keras.layers.Conv2DTranspose(16,kernel_size=3, strides=2,padding="same",
                                                                     activation="selu"),
                                        keras.layers.Conv2DTranspose(1,kernel_size=3, strides=2,padding="same",
                                                                     activation="sigmoid"),
                                        keras.layers.Reshape([28,28])
])

conv_ae = keras.models.Sequential([conv_encoder,conv_decoder])


conv_ae.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr=1.0),
                metrics=[rounded_accuracy])
history = conv_ae.fit(X_train, X_train, epochs=5,
                      validation_data=(X_valid, X_valid))

In [ ]:
conv_encoder.summary()
conv_decoder.summary()

In [ ]:
show_reconstructions(conv_ae)

# 17.5 순환 오토인코더

In [ ]:
recurrent_encoder = keras.models.Sequential([
                                             keras.layers.LSTM(100,return_sequences=True,
                                                               input_shape=[None,28]),
                                             keras.layers.LSTM(30)
])

recurrent_decoder =keras.models.Sequential([
                                            keras.layers.RepeatVector(28, input_shape=[30]),
                                            keras.layers.LSTM(100, return_sequences=True),
                                            keras.layers.TimeDistributed(keras.layers.Dense(28, activation="sigmoid"))
])

recurrent_ae = keras.models.Sequential([recurrent_encoder,recurrent_decoder])

# 17.6 잡음 제거 오토인코더

입력에 잡음을 추가하여 오토인코더가 유용한 특성을 학습하도록 강제

방법1. 입력에 가우시안노이즈

방법2. 드롭아웃

In [ ]:
 dropout_encoder = keras.models.Sequential([
                                            keras.layers.Flatten(input_shape=[28,28]),
                                            keras.layers.Dropout(0.5),
                                            keras.layers.Dense(100,activation="selu"),
                                            keras.layers.Dense(30,activation="selu")
 ])

 dropout_decoder = keras.models.Sequential([
                                            keras.layers.Dense(100,activation="selu",input_shape=[30]),
                                            keras.layers.Dense(28*28,activation="sigmoid"),
                                            keras.layers.Reshape([28,28])
 ])

 dropout_ae = keras.models.Sequential([dropout_encoder,
                                       dropout_decoder])

# 17.7 희소 오토인코더

l1규제 혹은

 코딩레이어에 시그모이드를 적용하여 특정 특성을 줄임.

In [ ]:
sparse_l1_encoder = keras.models.Sequential([
                                             keras.layers.Flatten(input_shape=[28,28]),
                                             keras.layers.Dense(100, activation="selu"),
                                             keras.layers.Dense(300, activation="sigmoid"),
                                             keras.layers.ActivityRegularization(l1=1e-3)
])

sparse_l1_decoder = keras.models.Sequential([
                                             keras.layers.Dense(100,activation="selu",input_shape=[300]),
                                             keras.layers.Dense(28*28,activation="sigmoid"),
                                             keras.layers.Reshape([28,28])
])


sparse_l1_ae = keras.models.Sequential([sparse_l1_encoder,sparse_l1_decoder])


각 뉴런에 대한 평균 활성화 정도를 계산하여

희소 손실을 추가하여 너무 활성화되거나 너무 활성화되지 않은 뉴런에 대해 규제를 할 수있음.

ex) 목표0.1 실제 0.3 일때 mse형태로 (0.3-0.1)^2를 손실에 추가 --> 규제됨






mae,mse도 가능한데 Kl divergence도 활용가능.

--> 뉴런이 활성화될 목표확률p와 실제확률 q사이의 발산

kl divergence 예시

In [ ]:
K = keras.backend
kl_divergence = keras.losses.kullback_leibler_divergence

class KLDivergenceRegularizer(keras.regularizers.Regularizer):
  def __init__(self, weight, target=0.1):
    self.weight = weight
    self.target = target
  def __call__(self,inputs):
    #활성화 평균
    mean_activities = K.mean(inputs,axis=0)
    return self.weight *(
        kl_divergence(self.target, mean_activities) +
        kl_divergence(1. -self.target, 1.-mean_activities)
    )

In [ ]:
kld_reg = KLDivergenceRegularizer(weight=0.05, target=0.1) #가중치 0.05 목표 0.1
sparse_kl_encoder = keras.models.Sequential([
                                             keras.layers.Flatten(input_shape=[28,28]),
                                             keras.layers.Dense(100,activation='selu'),
                                             keras.layers.Dense(300,activation='sigmoid',activity_regularizer=kld_reg)
])
sparse_kl_decoder = keras.models.Sequential([
                                             keras.layers.Dense(100,activation='selu',input_shape=[300]),
                                             keras.layers.Dense(28*28,activation="sigmoid"),
                                             keras.layers.Reshape([28,28])
])

sparse_kl_ae = keras.models.Sequential([sparse_l1_encoder,sparse_l1_decoder])

## 17.8 변이형 오토인코더

변분 베이즈 추론 사용하여 

가우시안 분포에서 코딩을 샘플링

기존의 재구성 오차에 

코딩의 분포를 가우시안 분포로 이동시키기 위한 잠재손실(latent loss) 사용

샘플링함수.

mean과 log_var(==log시그마^2)를 받아서 

코딩벡터를 샘플링해줌

-->mean+ var

In [ ]:
K = keras.backend

class Sampling(keras.layers.Layer):
  def call(self,inputs):
    mean, log_var = inputs
    return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

variational encoder

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)


codings_size=10

inputs = keras.layers.Input(shape=[28,28])
z=keras.layers.Flatten()(inputs)
z=keras.layers.Dense(150, activation="selu")(z)
z=keras.layers.Dense(100,activation="selu")(z)

#인코더는 코딩을 바로 만들지 않고 평균코딩과 표준편차를 만든다. --> kl-divergence 활용하여 정규분포로 근사시킴. --> backpropa로 구해질듯.
#mean과 log_var을 구하기 위해 같은 입력사용
codings_mean = keras.layers.Dense(codings_size)(z)
codings_log_var = keras.layers.Dense(codings_size)(z)
#mean과 log_var을 사용해 샘플링.
#앞서 정의한 샘플링함수 사용
codings = Sampling()([codings_mean, codings_log_var])

#인코더는 샘플링한 mean, log_var, 코딩을 출력
varidational_encoder = keras.Model(
    inputs=[inputs],
    #mean,var은 출력 필요없는데 확인용
    outputs=[codings_mean,codings_log_var,codings]
)

variational decoder

In [ ]:
decoder_inputs = keras.layers.Input(shape=[codings_size])
x = keras.layers.Dense(100,activation="selu")(decoder_inputs)
x=  keras.layers.Dense(150,activation="selu")(x)
x=  keras.layers.Dense(28*28,activation="sigmoid")(x)
outputs = keras.layers.Reshape([28,28])(x)
variational_decoder = keras.models.Model(inputs=[decoder_inputs],
                                         outputs=outputs)

In [ ]:
_,_,codings = varidational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = keras.models.Model(inputs=[inputs],outputs=[reconstructions])

latent loss (목표분포로 코딩분포를 이동시킴) --> kl divergence

In [ ]:
latent_loss = -0.5 * K.sum(
    1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
    axis=-1)

In [ ]:
variational_ae.add_loss(K.mean(latent_loss)/784.)

In [ ]:
variational_ae.compile(loss=keras.losses.binary_crossentropy,
                       optimizer=keras.optimizers.RMSprop(),
                       metrics =[rounded_accuracy])

history =variational_ae.fit(X_train,X_train,epochs=25,batch_size=128,
                            validation_data=(X_valid,X_valid))

In [ ]:
show_reconstructions(variational_ae)
plt.show()

17.8.1 패션 mnist 생성하기

In [ ]:
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

In [ ]:
codings = tf.random.normal(shape=[12, codings_size])
#12개의 평균,표준편차 샘플을 얻음.
images = variational_decoder(codings).numpy()
plot_multiple_images(images,4)

시맨틱 보간 가능.

픽셀 수준의 보간 대신 코딩 수준에서의 보간

픽셀수준이 아니라, 코딩을 reshape로 보간하여 디코더 통과시킴

In [ ]:

codings_grid = tf.reshape(codings, [1, 3, 4, codings_size])
#코딩을 더 큰 그리드로 보간시킴
larger_grid = tf.image.resize(codings_grid, size=[5, 7])
interpolated_codings = tf.reshape(larger_grid, [-1, codings_size])
#보간된 코딩을 디코더 통과시킴
images = variational_decoder(interpolated_codings).numpy()

plt.figure(figsize=(7, 5))
for index, image in enumerate(images):
    plt.subplot(5, 7, index + 1)
    if index%7%2==0 and index//7%2==0:
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
    else:
        plt.axis("off")
    plt.imshow(image, cmap="binary")
save_fig("semantic_interpolation_plot", tight_layout=False)

# 17.9 생성적 적대 신경망 GAN

훈련1. 판별자를 학습:  진짜, 가짜 이미지를 판별하도록 

훈련2. 생성자를 학습 : 판별자를 속이도록 학습(판별자 가중치 동결)

생성자. 

오토인코더의 디코더와 비슷

가우시안에서 샘플링하여 이미지 생성



In [ ]:
codings_size =30

generator = keras.models.Sequential([
                                     keras.layers.Dense(100,activation="selu",input_shape=[codings_size]),
                                     keras.layers.Dense(150,activation="selu"),
                                     keras.layers.Dense(28*28,activation="sigmoid"),
                                     keras.layers.Reshape([28,28])
])



In [ ]:
discriminator = keras.models.Sequential([
                                         keras.layers.Flatten(input_shape=[28,28]),
                                         keras.layers.Dense(150, activation="selu"),
                                         keras.layers.Dense(100, activation="selu"),
                                         keras.layers.Dense(1, activation="sigmoid")
])


gan = keras.models.Sequential([generator, discriminator])

In [ ]:
discriminator.compile(loss= keras.losses.binary_crossentropy,
                      optimizer = keras.optimizers.RMSprop())

discriminator.trainable=False

gan.compile(loss = keras.losses.binary_crossentropy,
            optimizer = keras.optimizers.RMSprop())

In [ ]:
batch_size=32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)

일반적 반복훈련이 아니기때문에 사용자 정의 훈련 사용

In [ ]:
def train_gan(gan,dataset,batch_size,codings_size,n_epochs=50):
  generator, discriminator = gan.layers #gan = keras.models.Sequential([generator, discriminator])
  for epoch in range(n_epochs):
    print("Epoch {}/{}".format(epoch+1,n_epochs))
    for X_batch in dataset:
      #훈련1. 판별자 학습
      #노이즈 생성 [배치사이즈,코딩크기]
      noise = tf.random.normal(shape = [batch_size,codings_size])
      #이미지 생성
      generated_images = generator.predict(noise)
      #진짜와 가짜 concat
      X_fake_and_real = tf.concat([X_batch,generated_images],axis=0)
      #labeling
      y1 = tf.constant([[1.]]*batch_size + [[0.]]*batch_size)
      discriminator.trainable=True
      discriminator.train_on_batch(X_fake_and_real,y1)
      
      
      #훈련2. 생성자 학습
      noise = tf.random.normal(shape=[batch_size,codings_size])
      y2 = tf.constant([[1.]]*batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise,y2)
    plot_multiple_images(generated_images,8)
    plt.show()




In [ ]:
train_gan(gan, dataset, batch_size, codings_size,n_epochs=1)
#에폭을 늘리면 더 좋아짐~

### 17.9.2 심층 합성곱 GAN

여러 파라미터와 층을 조절해 깊게 쌓음.

경우에 따라 여러 조합을 직접 해봐야 좋은 결과 가능

for mnist

배치정규화 ,전치합성곱 등 사용

In [ ]:
codings_size=100

generator = keras.models.Sequential([
                                     #100차원의 코딩을 받아 7*7*128 차원으로 투영
                                     keras.layers.Dense(7*7*128,input_shape=[codings_size]),
                                     #7,7,128 크기의 텐서로 변환
                                     keras.layers.Reshape([7,7,128]),
                                     #BN
                                     keras.layers.BatchNormalization(),
                                     #7*7에서 14*14로 업샘플링, 깊이는 128에서 64로
                                     keras.layers.Conv2DTranspose(64, kernel_size=5,strides=2, padding='same',
                                                                  activation="selu"),
                                     keras.layers.BatchNormalization(),
                                     #14*14에서 28*28로 업샘플링. 깊이는 64에서 1로
                                     keras.layers.Conv2DTranspose(1, kernel_size=5,strides=2,padding="same",
                                                                  activation="tanh")
])

#이진분류를 위한 일반적인 cnn구조와 흡사함.
#풀링대신 스트라이드 2 사용.
#생성자의 아웃풋에 맞게 X_train을 X_train.reshape(-1,28,28,1) 해야함.
discriminator = keras.models.Sequential([
                                         keras.layers.Conv2D(64,kernel_size=5,strides=2,padding="same",
                                                             activation=keras.layers.LeakyReLU(0.2),
                                                             input_shape = [28,28,1]),
                                         keras.layers.Dropout(0.4),
                                         keras.layers.Conv2D(128, kernel_size=5, strides=2,padding="same",
                                                             activation=keras.layers.LeakyReLU(0.2)),
                                         keras.layers.Dropout(0.4),
                                         keras.layers.Flatten(),
                                         keras.layers.Dense(1,activation="sigmoid")
])

## 연습문제 9. 잡음제거 오토인코더를 사용해 이미지 분류기를 사전훈련해보세요.

a. 데이터셋을 훈련 세트와 테스트로 나누세요

전체 훈련 세트에서 심층 잡음 제거 오토인코더를 훈련하세요

나누기

In [ ]:
(X_train_full,y_train_full),(X_test,y_test)  = keras.datasets.mnist.load_data()
X_train,X_valid = X_train_full[5000:]/255.,X_train_full[:5000]/255.
y_train,y_valid = y_train_full[5000:],y_train_full[:5000]
X_test = X_test/255.

In [ ]:
X_valid.shape

심층 잡음제거 오토인코더

In [ ]:
denoise_encoder = keras.models.Sequential([
                                           keras.layers.GaussianNoise(0.1, input_shape=[28, 28]),
                                           keras.layers.Flatten(input_shape=[28,28]),
                                           keras.layers.Dense(150,activation='selu'),
                                           keras.layers.Dense(100,activation="selu")
])

denoise_decoder = keras.models.Sequential([
                                           keras.layers.Dense(150,input_shape=[100],activation="selu"),
                                           keras.layers.Dense(28*28, activation="sigmoid"),
                                           keras.layers.Reshape([28,28])
])

In [ ]:
denoise_ae = keras.models.Sequential([denoise_encoder,denoise_decoder])
denoise_ae.compile(loss=  keras.losses.binary_crossentropy,
                   optimizer = keras.optimizers.Adam(),
                   metrics=[rounded_accuracy])

In [ ]:
history = denoise_ae.fit(X_train,X_train,epochs=20,
                    validation_data=(X_valid,X_valid))

b.이미지가 잘 재구성되는지 확인

In [ ]:
codings = denoise_encoder.predict(X_valid[:12])
label = y_valid[:12]

In [ ]:
label

In [ ]:
reconstruction = denoise_decoder(codings)

In [ ]:
for index, image in enumerate(reconstruction):
  plt.subplot(4,3,index+1)
  plt.imshow(image,cmap="binary")
  plt.title("true label: {}".format(label[index]))
  plt.axis("off")

In [ ]:
for index, image in enumerate(X_valid[:12]):
  plt.subplot(4,3,index+1)
  plt.imshow(image,cmap="binary")
  plt.title("true label: {}".format(label[index]))
  plt.axis("off")

c.사전훈련된 층을 활용해 훈련셋 500개로 성능비교

In [ ]:
X_train_small = X_train[:500]
y_train_small = y_train[:500]

사전훈련 없을떄.

In [ ]:
non_pretrained_clf = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(20, activation="selu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
non_pretrained_clf.compile(loss=keras.losses.sparse_categorical_crossentropy,
                           optimizer= keras.optimizers.Adam(),
                           metrics = 'accuracy')

In [ ]:
history = non_pretrained_clf.fit(X_train_small,y_train_small,
                                 epochs=20,
                                 validation_data = (X_valid,y_valid))

In [ ]:
non_pretrained_clf.evaluate(X_test,y_test)

비지도 사전훈련 활용

In [ ]:
denoise_encoder_clone = keras.models.clone_model(denoise_encoder)

In [ ]:
pretrained_clf = keras.models.Sequential([
    denoise_encoder_clone,
    keras.layers.Dense(20, activation="selu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
pretrained_clf.compile(loss=keras.losses.sparse_categorical_crossentropy,
                           optimizer= keras.optimizers.Adam(),
                           metrics = 'accuracy')

In [ ]:
history = pretrained_clf.fit(X_train_small,y_train_small,
                                 epochs=20,
                                 validation_data = (X_valid,y_valid))

In [ ]:
pretrained_clf.evaluate(X_test,y_test)

# 연습문제 10. 이미지 데이터셋을 하나 선택해 변이형 오토인코더를 훈련하고 이미지를 생성해보세요.

In [ ]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.cifar10.load_data()

In [ ]:
X_train,X_valid = X_train_full[5000:]/255., X_train_full[:5000]/255.
X_test = X_test/255.

In [ ]:
X_train.shape

In [ ]:
len(np.unique(y_test))

variational auto encoder

샘플링 레이어

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
K = keras.backend

class Sampling(keras.layers.Layer):
  def call(self,inputs):
    mean, log_var = inputs
    return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

인코더

In [ ]:
codings_size=75
inputs = keras.layers.Input(shape=[32,32,3])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(300,activation = 'selu')(z)
z = keras.layers.Dense(150,activation = "selu")(z)

mean  =keras.layers.Dense(codings_size)(z)
log_var = keras.layers.Dense(codings_size)(z)

codings = Sampling()([mean,log_var])

variational_encoder = keras.Model(
                                   inputs = [inputs],
                             
                                   outputs = [mean,log_var,codings]
)

디코더

In [ ]:
decoder_inputs = keras.layers.Input(shape=[codings_size])
x = keras.layers.Dense(150,activation="selu")(decoder_inputs)
x = keras.layers.Dense(300,activation="selu")(x)
x = keras.layers.Dense(32*32*3, activation="selu")(x)
outputs = keras.layers.Reshape([32,32,3])(x)

variational_decoder = keras.models.Model(
                                   inputs = decoder_inputs,
                                   outputs = outputs
)

                      

In [ ]:
means,log_vars,codings = variational_encoder(inputs)
reconstruction = variational_decoder(codings)
variational_ae = keras.models.Model(inputs = [inputs], outputs = [reconstruction])

잠재 손실

In [ ]:
latent_loss = -0.5 * K.sum(
    1 + log_var - K.exp(log_var) - K.square(mean),
    axis=-1)

In [ ]:
variational_ae.add_loss(K.mean(latent_loss)/1024*3.)

In [ ]:
variational_ae.compile(loss=keras.losses.mean_squared_error,
                       optimizer=keras.optimizers.RMSprop(lr=0.001),
                       metrics =[rounded_accuracy])

history =variational_ae.fit(X_train,X_train,epochs=25,batch_size=128,
                            validation_data=(X_valid,X_valid))

이미지 생성해보기

In [ ]:
codings = tf.random.normal(shape = [12,codings_size])
reconstructions = variational_decoder(codings)
plot_multiple_images(reconstructions,4)

단순 ae로도 도전.

In [ ]:
encoder = keras.models.Sequential([
                                   keras.layers.Flatten(input_shape=[32,32,3]),
                                   keras.layers.Dense(300,activation="selu"),
                                   keras.layers.Dense(150,activation="selu"),
                                   keras.layers.Dense(50,activation='selu'),
])

decoder = keras.models.Sequential([
                                   keras.layers.Dense(150,input_shape=[50],activation='selu'),
                                   keras.layers.Dense(300,activation="selu"),
                                   keras.layers.Dense(32*32*3, activation="sigmoid"),
                                   keras.layers.Reshape([32,32,3])

])

ae = keras.models.Sequential([encoder,decoder])

In [ ]:
ae.compile(loss=keras.losses.binary_crossentropy,
           optimizer=keras.optimizers.RMSprop(),
           metrics=[rounded_accuracy])
history = ae.fit(X_train,X_train,
                 epochs=20,
                 validation_data=(X_valid,X_valid))

In [ ]:
n_images = 12
new_images = X_test[:n_images]

recons= ae.predict(new_images)

In [ ]:
plot_multiple_images(recons,4)

# 연습문제 11 DCGAN을 사용하여 이미지를 트레이닝하고 생성해보기

In [ ]:
(X_train_all,y_train_all),(X_test,y_test) =keras.datasets.cifar10.load_data()
X_train,X_valid= X_train_all[5000:]/255., X_train_all[:5000]/255.
X_test = X_test/255.
y_train,y_valid = y_train_all[5000:],y_train_all[:5000]

In [ ]:
X_train.shape

dcgan

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

codings_size=100
#생성기
generator = keras.models.Sequential([
    keras.layers.Dense(8 * 8  * 128, input_shape=[codings_size]),
    keras.layers.Reshape([8 , 8 , 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="SAME",
                                 activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding="SAME",
                                 activation="tanh"),
])

#판별기
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2),
                        input_shape=[32, 32, 3]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")
])
gan = keras.models.Sequential([generator, discriminator])

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
X_train_dcgan = X_train.reshape(-1, 32, 32, 3) * 2. - 1. # reshape and rescale


In [ ]:
X_train_dcgan.shape

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train_dcgan)
dataset = dataset.shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [ ]:
def train_gan(gan,dataset,batch_size,codings_size,n_epochs=50):
  generator, discriminator = gan.layers #gan = keras.models.Sequential([generator, discriminator])
  for epoch in range(n_epochs):
    print("Epoch {}/{}".format(epoch+1,n_epochs))
    for X_batch in dataset:
      #훈련1. 판별자 학습
      #노이즈 생성 [배치사이즈,코딩크기]
      noise = tf.random.normal(shape = [batch_size,codings_size])
      #이미지 생성
      generated_images = generator.predict(noise)
      #진짜와 가짜 concat
      X_fake_and_real = tf.concat([X_batch,generated_images],axis=0)
      #labeling
      y1 = tf.constant([[1.]]*batch_size + [[0.]]*batch_size)
      discriminator.trainable=True
      discriminator.train_on_batch(X_fake_and_real,y1)
      
      
      #훈련2. 생성자 학습
      noise = tf.random.normal(shape=[batch_size,codings_size])
      y2 = tf.constant([[1.]]*batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise,y2)
    plot_multiple_images(generated_images,8)
    plt.show()


In [ ]:
train_gan(gan, dataset, batch_size, codings_size,n_epochs=10)